In [1]:
import json
import os

In [2]:
data = []
with open('prepared-wikipedia-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l['text'])

In [3]:
len(data)

97963

In [4]:
# !rm -rf generate-wikipedia-qa-llama3
!mkdir generate-wikipedia-qa-llama3

mkdir: cannot create directory ‘generate-wikipedia-qa-llama3’: File exists


In [5]:
results = []
for i in range(len(data)):
    
    s = """
```
{{replace_me}}
```

Generate list of QA choice in malay, return in JSON [{'question', 'A', 'B', 'C', 'D', 'answer'}]
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

97963

In [6]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = f'generate-wikipedia-qa-llama3/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 2048,
                }
                response = requests.post(
                    'https://llama-3.us.mesolitica.com/v1/chat/completions', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [7]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [8]:
urls = [(q, no) for no, q in enumerate(results)]

In [9]:
answer(*urls[10005])

In [10]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 98%|██████████████████████████████████████████████████████████████████████▌ | 96022/97963 [31:13:03<25:29,  1.27it/s]

In [ ]:
from glob import glob

files = glob('generate-wikipedia-qa/*.json')
len(files)

In [ ]:
files[0]

In [ ]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [ ]:
d

In [ ]:
json.loads(d[0])

In [ ]:
data[77747]

In [ ]:
data = []
with open('prepared-wikipedia.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l)

In [ ]:
set(q.keys())

In [ ]:
suppose = {'question', 'A', 'B', 'C', 'D', 'answer'}
len(set(q.keys()) | suppose)

In [ ]:
with open('generated-wikipedia-qa.jsonl', 'w') as fopen_j:
    for i in tqdm(range(len(data))):
        f = f'generate-wikipedia-qa/{i}.json'
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            d = json.load(fopen)
        
        questions = []
        unique_questions = set()
        for d_ in d:
            try:
                qa = json.loads(d_)['arguments']['qa']
                for q in qa:
                    if len(set(q.keys()) | suppose) != len(suppose):
                        continue
                    if q['question'].lower() not in unique_questions:
                        if not ('A' in q and isinstance(q['A'], str)):
                            continue
                        if not ('B' in q and isinstance(q['B'], str)):
                            continue
                        if not ('C' in q and isinstance(q['C'], str)):
                            continue
                        if not ('D' in q and isinstance(q['D'], str)):
                            continue
                        if not ('answer' in q and isinstance(q['answer'], str)):
                            continue
                            
                        for k in suppose:
                            q[k] = q[k].encode("utf8", errors="surrogatepass").decode("utf8")

                        questions.append(q)
                        unique_questions.add(q['question'].lower())
            except:
                pass
        
        data[i]['text'] = data[i]['text'].encode("utf8", errors="surrogatepass").decode("utf8")
        data[i]['questions'] = questions
        fopen_j.write(f'{json.dumps(data[i])}\n')

In [ ]:
!wc -l generated-wikipedia-qa.jsonl

In [ ]:
!ls -lh generated-wikipedia-qa.jsonl

In [ ]:
d = []
with open('generated-wikipedia-qa.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        
        d.append(l)

In [ ]:
from datasets import Dataset

In [ ]:
dataset = Dataset.from_list(d)

In [ ]:
dataset.push_to_hub('mesolitica/mallam-small-malaysian-qa-choice', split = 'wikipedia')